# Basic pre-assembled models
In this section we train the basic models which Ultralytics provides per default. 
All of those models come with COCO weights and thus benefit from Transfer Learning and have a good baseline performance. 


In [1]:
import os

CURRENT_DIR = os.getcwd()
PROJECT_NAME = 'Unity_Eyes'

def get_basic_settings(name, epochs, project_name=PROJECT_NAME):
    train = f'{CURRENT_DIR}/yolov5/yolov5/train.py'
    data = f'--data {CURRENT_DIR}/yolov5/yolov5/unity_eyes.yaml'
    output = f'--project {CURRENT_DIR}/models/{PROJECT_NAME} --name {name}'
    
    settings = f'--imgsz 320 --epochs {epochs}'
    
    return train, data, output, settings

In [2]:
def train_basic_xtra_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Xtra', epochs)    
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 16 --weights yolov5x.pt
        
        
def train_basic_large_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Large', epochs)    
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 16 --weights yolov5l.pt
        
        
def train_basic_medium_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Medium', epochs)    
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 16 --weights yolov5m.pt
        
        
def train_basic_small_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Small', epochs)    
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 16 --weights yolov5s.pt
        
        
def train_basic_nano_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Nano', epochs)    
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 16 --weights yolov5n.pt

In [3]:
if False:
    train_basic_nano_model(10)
    train_basic_small_model(10)
    train_basic_medium_model(10)
    train_basic_large_model(10)
    train_basic_xtra_model(10)

## Analyzing Batch Sizes

In [14]:
PROJECT_NAME = 'Batch_Sizes'

def train_nano_model_batch(repitions, batch_size=16, epochs=20): 
    train, data, output, settings = get_basic_settings(f'Nano_Batch_{batch_size}', epochs, 
                                                       project_name=f'{PROJECT_NAME}_Batch')    
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch {batch_size} --weights yolov5n.pt
        

In [5]:
if False:
    train_nano_model_batch(4, batch_size=256)
    train_nano_model_batch(4, batch_size=128)
    train_nano_model_batch(4, batch_size=64)
    train_nano_model_batch(4, batch_size=32)
    train_nano_model_batch(4, batch_size=16)
    train_nano_model_batch(4, batch_size=8)
    
    train_nano_model_batch(2, batch_size=256)
    train_nano_model_batch(2, batch_size=128)
    train_nano_model_batch(2, batch_size=64)
    train_nano_model_batch(2, batch_size=32)
    train_nano_model_batch(2, batch_size=16)
    train_nano_model_batch(2, batch_size=8)

|Batch|Memory|Time Epoch|mAP.95|
|-----|----- |----------|------|
|256  |6.04G |14.57min  |96.87%|
|128  |4.62G |17.54min  |97.11%|
|64   |2.25G |19.03min  |97.26%|
|32   |1.16G |19.52min  |97.37%|
|16   |0.60G |44.09min  |97.09%|
|8    |0.28G |64.63min  |97.11%|


# First WandB Sweep - Random Search 

In [6]:
import wandb

sweep_config = {
    "program" : f'{CURRENT_DIR}/yolov5/yolov5/train_wrapper.py',
    "project" : f'{CURRENT_DIR}/models/Unity_Nano_Sweep',
    "name" : "Nano_Sweep",
    "method" : "random",
    "metric": {
      "goal": "maximize",
      "name": "metrics/mAP_0.5:0.95"},
    "parameters" : {   
      "batch_size": { "max": 32, "min": 16, "distribution": "int_uniform"},
      "weight_decay": { "max": 0.001, "min": 0.00025, "distribution": "uniform"},
      "translate": { "max": 0.2, "min": 0.05, "distribution": "uniform"},
      "momentum": { "max": 1.0, "min": 0.4685, "distribution": "uniform"},
      "fliplr": { "max": 1, "min": 0.25, "distribution": "uniform"},
      "scale": { "max": 1, "min": 0.25, "distribution": "uniform"},
      "iou_t": { "max": 0.4, "min": 0.1, "distribution": "uniform"},
      "hsv_v": { "max": 0.8, "min": 0.2, "distribution": "uniform"},
      "hsv_s": { "max": 1.4, "min": 0.35, "distribution": "uniform"},
      "hsv_h": { "max": 0.03, "min": 0.0075, "distribution": "uniform"},
      "lrf": { "max": 0.2, "min": 0.05, "distribution": "uniform"},
      "lr0": { "max": 0.02, "min": 0.005, "distribution": "uniform"},
      "cls": { "max": 1, "min": 0.25, "distribution": "uniform"},
      "box": { "max": 0.1, "min": 0.025, "distribution": "uniform"}
    }
}

if False:
    sweep_id = wandb.sweep(sweep_config)

In [7]:
if False:
    !wandb agent daniel-kamphake/uncategorized/w5nirba8

* Batch # 16-32
* translate # 0.05 - 0.2
* fliplr # 0.25-1
* scale 
* momentum # 0.4685 - 1.0
* hsv_h # 0.0075 - 0.03
* lr0 # 0.005 - 0.02 
* cls # 0.25-1
* box # 0.025 - 0.1

# Second WandB Sweep - Bayesian Search

In [8]:
import wandb

sweep_config_new = {
    "program" : f'{CURRENT_DIR}/yolov5/yolov5/train_wrapper.py',
    "project" : f'{CURRENT_DIR}/models/Unity_Nano_Sweep_2',
    "name" : "Nano_Sweep_2",
    "method" : "bayes",
    "metric": {
      "goal": "maximize",
      "name": "metrics/mAP_0.5:0.95"},
    "parameters" : {   
      "batch_size": { "max": 40, "min": 24, "distribution": "int_uniform"},
      "weight_decay": { "max": 0.001, "min": 0.00025, "distribution": "uniform"},
      "translate": { "max": 0.1, "min": 0.0, "distribution": "uniform"},
      "fliplr": { "max": 0.6, "min": 0.4, "distribution": "uniform"},
      "scale": { "max": 0.6, "min": 0.4, "distribution": "uniform"},
      "momentum": { "max": 0.9, "min": 0.7, "distribution": "uniform"},
      "hsv_v": { "max": 0.8, "min": 0.2, "distribution": "uniform"},
      "hsv_s": { "max": 1.4,  "min": 0.35, "distribution": "uniform"},
      "hsv_h": { "max": 0.025, "min": 0.01, "distribution": "uniform"},
      "lr0": { "max": 0.02, "min": 0.0125, "distribution": "uniform"},
      "cls": { "max": 0.5, "min": 0.2, "distribution": "uniform"},
      "box": { "max": 0.125, "min": 0.05, "distribution": "uniform"}
    }
}

if False:
    sweep_id = wandb.sweep(sweep_config_new)

In [9]:
if False:
    !wandb agent daniel-kamphake/uncategorized/12chmrr7

# Train with specific configuration 

In [11]:
Path = f'/home/daniel/Seafile/Masterarbeit/pupil-detection/yolov5/yolov5/train_wrapper.py'
batch = f'--batch_size=40'
loss = f'--box=0.123 --cls=0.2'
image = f'--hsv_h=0.014 --hsv_v=0.37 --hsv_s=1.2 --shear=0.0 --degrees=0.0'
augments = f'--fliplr=0.43 --scale=0.54 --translate=0.08'
learn_rate = f'--lrf=0.02 --lr0=0.0165 --momentum=0.89  --weight_decay=0.0009 --iou_t=0.017' 
data = f'--data {CURRENT_DIR}/yolov5/yolov5/unity_eyes.yaml'
output = f'--project {CURRENT_DIR}/models/{PROJECT_NAME} --name Nano_opt'
settings = f'--imgsz 320 --epochs 20 --weights yolov5n.pt'

command = f'python {Path} {data} {output} {settings} {batch} {loss} {image} {augments} {learn_rate}'
print(command)

python /home/daniel/Seafile/Masterarbeit/pupil-detection/yolov5/yolov5/train_wrapper.py --data /home/daniel/Seafile/Masterarbeit/pupil-detection/yolov5/yolov5/unity_eyes.yaml --project /home/daniel/Seafile/Masterarbeit/pupil-detection/models/Unity_Eyes --name Nano_opt --imgsz 320 --epochs 20 --weights yolov5n.pt --batch_size=40 --box=0.123 --cls=0.2 --hsv_h=0.014 --hsv_v=0.37 --hsv_s=1.2 --shear=0.0 --degrees=0.0 --fliplr=0.43 --scale=0.54 --translate=0.08 --lrf=0.02 --lr0=0.0165 --momentum=0.89  --weight_decay=0.0009 --iou_t=0.017


In [ ]:
if True:
    for _ in range(20):
        !{command}

wandb: Currently logged in as: daniel-kamphake (use `wandb login --relogin` to force relogin)
train_wrapper: weights=yolov5n.pt, cfg=, data=/home/daniel/Seafile/Masterarbeit/pupil-detection/yolov5/yolov5/unity_eyes.yaml, hyp=/home/daniel/Seafile/Masterarbeit/pupil-detection/yolov5/yolov5/data/hyps/sweep_hyper.yaml, epochs=20, batch_size=40, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=/home/daniel/Seafile/Masterarbeit/pupil-detection/models/Unity_Eyes, name=Nano_opt, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, lr0=0.0165, lrf=0.02, cls=0.2, box=0.123, momentum=0.89, weight_decay=0.0009, iou_t=0.017, hsv_v=0.37, hsv_s=1.2, hsv_h=0.014, fliplr=0.43, scale

               Class     Images     Labels          P          R     mAP@.5 mAP@

# Weightless Training with Nano and Micro
In this section we reduce the size of the Nano model by decreasing the scaling factors in the `.yaml` file, which describes the architecture of each model.


In [35]:
PROJECT_NAME = 'Weightless'

def get_weightless_settings(name, epochs, project_name=PROJECT_NAME):
    train = f'{CURRENT_DIR}/yolov5/yolov5/train.py'
    data = f'--data {CURRENT_DIR}/yolov5/yolov5/unity_eyes.yaml'
    output = f'--project {CURRENT_DIR}/models/{PROJECT_NAME} --name {name}'
    
    settings = f'--imgsz 320 --epochs {epochs}'
    
    return train, data, output, settings


def train_weightless_nano_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Nano', epochs)   
    cfg = f'--cfg {CURRENT_DIR}/yolov5/yolov5/models/yolov5n.yaml'
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 40 {cfg}
                

def train_weightless_nano_shallow_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Shallow', epochs)   
    cfg = f'--cfg {CURRENT_DIR}/yolov5/yolov5/models/yolov5n_shallow.yaml'
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 40 {cfg}
                

def train_weightless_nano_thin_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Thin', epochs)   
    cfg = f'--cfg {CURRENT_DIR}/yolov5/yolov5/models/yolov5n_thin.yaml'
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 40 {cfg}
                            

def train_weightless_micro_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Micro', epochs)   
    cfg = f'--cfg {CURRENT_DIR}/yolov5/yolov5/models/yolov5micro.yaml'
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 40 {cfg}
        

def train_weightless_super_micro_model(repitions, epochs=20): 
    train, data, output, settings = get_basic_settings('Super Micro', epochs)   
    cfg = f'--cfg {CURRENT_DIR}/yolov5/yolov5/models/yolov5micro_super.yaml'
    for i in range(repitions):
        !python {train} {data} {settings} {output} --batch 40 {cfg}

In [36]:
if False:
    train_weightless_nano_model(10)
    train_weightless_nano_shallow_model(10)
    train_weightless_nano_thin_model(10)
    train_weightless_micro_model(10)    
    train_weightless_super_micro_model(10)

## Nano
Model Summary: 270 layers, 1766623 parameters, 1766623 gradients, 4.2 GFLOPs

20 epochs completed in 0.276 hours.

## Shallow
Model Summary: 243 layers, 1673823 parameters, 1673823 gradients, 3.8 GFLOPs

20 epochs completed in 0.257 hours.

## Thin
Model Summary: 270 layers, 329207 parameters, 329207 gradients, 1.0 GFLOPs

20 epochs completed in 0.240 hours.

## Micro
Model Summary: 243 layers, 310679 parameters, 310679 gradients, 0.9 GFLOPs

20 epochs completed in 0.241 hours.

## Super Micro
Model Summary: 192 layers, 94651 parameters, 94651 gradients, 0.4 GFLOPs